In [1]:
# Import libraries
import datetime
from tabulate import tabulate

In [2]:
# Create pricing data variables in nested dictionary
pricing = {
    'Ladies': {
        'Standard': {
            'Dress': 8.00,
            '2 PC Suit': 12.00,
            'Jacket': 7.50,
            'Shirt': 6.50,
            'Blouse': 6.50
        },
        'Specialist': {
            'Dress': 11.20,
            'Eve Dress': 21.00,
            '2 PC Suit': 16.80,
            'Jacket': 9.10,
            'Shirt': 9.10,
            'Blouse': 9.10
        }
    },
    'Gentlemen': {
        'Standard': {
            '2 PC Suit': 12.00,
            'Jacket': 7.50,
            'Trousers': 6.50,
            'Shirt': 6.50,
            'Coat': 9.95
        },
        'Specialist': {
            '2 PC Suit': 16.80,
            'Jacket': 10.50,
            'Trousers': 9.10,
            'Shirt': 9.10,
            'Coat': 13.93
        }
    }
}       

In [3]:
# VAT at 20%
VAT_RATE = 0.20

In [4]:
# Calculate amount of discount (If Applicable)
def calculate_discount(total):
    if total > 30:
        return 0.15 # 15% Discount on orders over £30
    elif total > 15:
        return 0.10 # 10% Discount on orders over £15
    else:
        return 0 # No discount

In [5]:
# Normalize Currency
def format_currency(value):
    return f"£{value: .2f}"

In [6]:
# Create tag and code with datetime
def generate_tag(date_time, service_type):
    return date_time.strftime('%d%m%Y%H%M') + '-' + ('st' if service_type == 'Standard' else 'sp')

In [7]:
# Display the menu
def display_menu():
    print("Welcome to Chi Cleaners Point of Sale System")
    print("===========================================")
    print("Price List")
    print("===========================================")
    for category, services in pricing.items():
        print(f"\n{category} Services")
        print("-" * (len(category) + 9))
        table = []
        headers = ["Item", "Standard Price", "Specialist Price"]
        items = set(pricing[category]['Standard'].keys()).union(pricing[category]['Specialist'].keys())
        for item in items:
            standard_price = pricing[category]['Standard'].get(item, "-")
            specialist_price = pricing[category]['Specialist'].get(item, "-")
            table.append([item, format_currency(standard_price) if isinstance(standard_price, float) else "-", 
                                 format_currency(specialist_price) if isinstance(specialist_price, float) else "-"])
        print(tabulate(table, headers, tablefmt="grid"))
    print("===========================================\n")

In [8]:
# Function to add item to order
def add_item_to_order(customer_items):
    print("Select Category:")
    print("1. Ladies")
    print("2. Gentlemen")
    category_choice = input("Enter choice (1/2): ")
    category = 'Ladies' if category_choice == '1' else 'Gentlemen'
    
    print("Select Service Type:")
    print("1. Standard")
    print("2. Specialist")
    service_choice = input("Enter choice (1/2): ")
    service_type = 'Standard' if service_choice == '1' else 'Specialist'
    
    print("Select Item:")
    items = list(pricing[category][service_type].keys())
    for i, item in enumerate(items, 1):
        print(f"{i}. {item}")
    item_choice = int(input("Enter choice: ")) - 1
    item_name = items[item_choice]
    item_price = pricing[category][service_type][item_name]
    
    customer_items.append((category, service_type, item_name, item_price))
    
    print(f"Added {item_name} ({category} - {service_type}) - {format_currency(item_price)} to order.\n")

In [9]:
# Generate the users receipt
def generate_receipt(customer_items):
    total_cost = sum(item[3] for item in customer_items)
    discount = calculate_discount(total_cost)
    discount_amount = total_cost * discount
    total_after_discount = total_cost - discount_amount
    vat = total_after_discount * VAT_RATE
    total_with_vat = total_after_discount + vat
    
    with open('receipt.txt', 'w') as receipt_file:
        receipt_file.write("Chi Cleaners Receipt\n")
        receipt_file.write("====================\n")
        for item in customer_items:
            receipt_file.write(f"{item[2]} ({item[0]} - {item[1]}) - {format_currency(item[3])}\n")
        receipt_file.write(f"Total: {format_currency(total_cost)}\n")
        receipt_file.write(f"Discount: {discount * 100}% (-{format_currency(discount_amount)})\n")
        receipt_file.write(f"Total after discount: {format_currency(total_after_discount)}\n")
        receipt_file.write(f"VAT (20%): {format_currency(vat)}\n")
        receipt_file.write(f"Total with VAT: {format_currency(total_with_vat)}\n")
    
    print("Receipt has been generated and saved to receipt.txt\n")

In [10]:
# Generate Tags
def generate_tags(customer_items):
    date_time = datetime.datetime.now()
    with open('tag.txt', 'w') as tag_file:
        for item in customer_items:
            tag = generate_tag(date_time, item[1])
            tag_file.write(f"Item: {item[2]}, Service: {item[1]}, Tag: {tag}\n")
    
    print("Tags have been generated and saved to tag.txt\n")

In [11]:
# Main loop (keeping track of customer items)
def main():
    customer_items = []
    
    display_menu()
    
    while True:
        add_item_to_order(customer_items)
        more_items = input("Add more items? (yes/no): ")
        if more_items.lower() != 'yes':
            break
    
    generate_receipt(customer_items)
    generate_tags(customer_items)
    print("Process completed. Check receipt.txt and tag.txt for details.")

In [ ]:
# Main
if __name__ == "__main__":
    main()

Welcome to Chi Cleaners Point of Sale System
Price List

Ladies Services
---------------
+-----------+------------------+--------------------+
| Item      | Standard Price   | Specialist Price   |
+===========+==================+====================+
| Shirt     | £ 6.50           | £ 9.10             |
+-----------+------------------+--------------------+
| Jacket    | £ 7.50           | £ 9.10             |
+-----------+------------------+--------------------+
| 2 PC Suit | £ 12.00          | £ 16.80            |
+-----------+------------------+--------------------+
| Dress     | £ 8.00           | £ 11.20            |
+-----------+------------------+--------------------+
| Blouse    | £ 6.50           | £ 9.10             |
+-----------+------------------+--------------------+
| Eve Dress | -                | £ 21.00            |
+-----------+------------------+--------------------+

Gentlemen Services
------------------
+-----------+------------------+--------------------+
| Item  